In [2]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from modules.feature_extractor import SIFT

# --- Load dataset info ---
with open('office_dataset_aruco/ground_truth_poses.json', 'r') as f:
    ground_truth = json.load(f)



camera_intrinsics = ground_truth["camera_intrinsics"]
K = np.array([[camera_intrinsics["fx"], 0, camera_intrinsics["cx"]],
              [0, camera_intrinsics["fy"], camera_intrinsics["cy"]],
              [0, 0, 1]], dtype=np.float32)



In [3]:
# Load Meshroom's reconstruction
with open('/home/leroy-marewangepo/Masters_Stuff/mono_vo/StructureFromMotion/output/cameras.sfm', 'r') as f:
    meshroom_data = json.load(f)

print("Meshroom data structure:")
print(f"Keys: {meshroom_data.keys()}")

# Explore the structure
if 'views' in meshroom_data:
    print(f"Number of views: {len(meshroom_data['views'])}")
    print(f"First view keys: {meshroom_data['views'][0].keys() if meshroom_data['views'] else 'None'}")

if 'poses' in meshroom_data:
    print(f"Number of poses: {len(meshroom_data['poses'])}")
    
if 'intrinsics' in meshroom_data:
    print(f"Number of intrinsics: {len(meshroom_data['intrinsics'])}")

if 'structure' in meshroom_data:
    print(f"Number of 3D points: {len(meshroom_data['structure'])}")
    print(f"First 3D point keys: {meshroom_data['structure'][0].keys() if meshroom_data['structure'] else 'None'}")

Meshroom data structure:
Keys: dict_keys(['version', 'featuresFolders', 'matchesFolders', 'views', 'intrinsics', 'poses'])
Number of views: 121
First view keys: dict_keys(['viewId', 'poseId', 'frameId', 'intrinsicId', 'resectionId', 'path', 'width', 'height', 'metadata'])
Number of poses: 121
Number of intrinsics: 1


In [4]:
# We want frames 1, 5, 10, 15, 20 for map, frame 25 for query
frames_we_need = [1, 5, 10, 15, 20, 25]

# Build mapping from frame number to pose
frame_to_pose = {}

for view in meshroom_data['views']:
    frame_id = int(view['frameId'])
    pose_id = view['poseId']
    
    if frame_id in frames_we_need:
        # Find the pose with this ID
        for pose in meshroom_data['poses']:
            if pose['poseId'] == pose_id:
                # Extract rotation matrix (9 elements -> 3x3)
                rot_flat = [float(x) for x in pose['pose']['transform']['rotation']]
                R = np.array(rot_flat).reshape(3, 3)
                
                # Extract camera center
                center = np.array([float(x) for x in pose['pose']['transform']['center']])
                
                frame_to_pose[frame_id] = {
                    'R': R,  # Camera-to-world rotation
                    'center': center,  # Camera position in world
                    'pose_id': pose_id
                }
                break

print("Extracted Meshroom poses:")
for frame_id in sorted(frame_to_pose.keys()):
    center = frame_to_pose[frame_id]['center']
    print(f"  Frame {frame_id}: position = {center}")

print(f"\nFound {len(frame_to_pose)} out of {len(frames_we_need)} frames")

Extracted Meshroom poses:
  Frame 1: position = [ -1.32500464 -14.09570895   0.55971041]
  Frame 5: position = [ -1.89106643 -14.03397593   0.48451178]
  Frame 10: position = [ -0.89950241 -14.06218714  -0.15353726]
  Frame 15: position = [ -0.540854   -14.04679548  -0.4511625 ]
  Frame 20: position = [ -0.11677747 -14.03355539  -0.65342604]
  Frame 25: position = [ -0.44401711 -14.03072321  -0.47325309]

Found 6 out of 6 frames


In [5]:
# We have Meshroom's camera poses in a unified coordinate system
# Now let's triangulate using those poses instead of estimating them

# Load the images we extracted features from earlier
import os

images_path = "office_dataset_aruco/left"

def load_image(frame_num):
    filename = f"frame_{frame_num:04d}.png"
    filepath = os.path.join(images_path, filename)
    img = cv2.imread(filepath)
    if img is None:
        return None
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

from modules.feature_extractor import SIFT

map_frames = [1, 5, 10, 15, 20]
query_frame = 25

# Extract features
extractor = SIFT(n_features=2000)
features = {}

for frame_num in map_frames + [query_frame]:
    img = load_image(frame_num)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        kp, desc = extractor.detect_and_compute(gray)
        features[frame_num] = {
            'keypoints': kp,
            'descriptors': desc,
            'image': img
        }
        print(f"Frame {frame_num}: {len(kp)} keypoints")

Frame 1: 1601 keypoints
Frame 5: 2000 keypoints
Frame 10: 1437 keypoints
Frame 15: 1491 keypoints
Frame 20: 1233 keypoints
Frame 25: 1194 keypoints


In [7]:
# Match pairs using OpenCV's findFundamentalMat with RANSAC
frame_pairs = [(1, 5), (1, 10), (5, 10), (10, 15), (15, 20)]

all_pair_results = {}

for frame_a, frame_b in frame_pairs:
    print(f"\n=== Pair {frame_a}-{frame_b} ===")
    
    # Match features
    desc_a = features[frame_a]['descriptors']
    desc_b = features[frame_b]['descriptors']
    matches = extractor.match_features(desc_a, desc_b)
    
    # Extract matched points
    pts_a = []
    pts_b = []
    match_info = []
    
    for match in matches:
        pt_a = features[frame_a]['keypoints'][match.queryIdx].pt
        pt_b = features[frame_b]['keypoints'][match.trainIdx].pt
        
        # Distance filter
        if np.linalg.norm(np.array(pt_b) - np.array(pt_a)) <= 700:
            pts_a.append(pt_a)
            pts_b.append(pt_b)
            match_info.append({
                'kp_idx_a': match.queryIdx,
                'kp_idx_b': match.trainIdx
            })
    
    pts_a = np.array(pts_a)
    pts_b = np.array(pts_b)
    
    print(f"  {len(pts_a)} filtered matches")
    
    # OpenCV RANSAC
    F, mask = cv2.findFundamentalMat(pts_a, pts_b, cv2.FM_RANSAC, 3.0, 0.99)
    
    if mask is not None:
        inlier_mask = mask.ravel() == 1
        inlier_pts_a = pts_a[inlier_mask]
        inlier_pts_b = pts_b[inlier_mask]
        inlier_indices = np.where(inlier_mask)[0]
        
        print(f"  {len(inlier_pts_a)} RANSAC inliers")
        
        all_pair_results[f"{frame_a}-{frame_b}"] = {
            'pts_a': inlier_pts_a,
            'pts_b': inlier_pts_b,
            'match_info': [match_info[i] for i in inlier_indices],
            'frame_a': frame_a,
            'frame_b': frame_b
        }
    else:
        print("  RANSAC failed")

print(f"\nProcessed {len(all_pair_results)} pairs successfully")


=== Pair 1-5 ===
  202 filtered matches
  31 RANSAC inliers

=== Pair 1-10 ===
  100 filtered matches
  55 RANSAC inliers

=== Pair 5-10 ===
  34 filtered matches
  23 RANSAC inliers

=== Pair 10-15 ===
  182 filtered matches
  124 RANSAC inliers

=== Pair 15-20 ===
  186 filtered matches
  64 RANSAC inliers

Processed 5 pairs successfully


In [10]:
# Build projection matrices from Meshroom poses
def build_P_from_meshroom(frame_id, K):
    pose = frame_to_pose[frame_id]
    R_c2w = pose['R']  # Camera-to-world
    center = pose['center']  # Camera position in world
    
    # Convert to world-to-camera for projection matrix
    R_w2c = R_c2w.T
    t_w2c = -R_w2c @ center
    
    return K @ np.hstack([R_w2c, t_w2c.reshape(-1, 1)])

# Triangulate all pairs using Meshroom poses
all_triangulated = {}

for pair_name, result in all_pair_results.items():
    print(f"\nTriangulating {pair_name}")
    
    frame_a = result['frame_a']
    frame_b = result['frame_b']
    
    # Build projection matrices from Meshroom
    P_a = build_P_from_meshroom(frame_a, K)
    P_b = build_P_from_meshroom(frame_b, K)
    
    pts_a_2d = result['pts_a']
    pts_b_2d = result['pts_b']
    
    # Triangulate
    points_4d = cv2.triangulatePoints(P_a, P_b, pts_a_2d.T, pts_b_2d.T)
    points_3d = points_4d[:3] / points_4d[3]
    points_3d = points_3d.T
    
    # Check reprojection error and positive depth
    valid_points = []
    valid_indices = []
    
    for i, pt in enumerate(points_3d):
        pt_hom = np.append(pt, 1)
        
        depth_a = (P_a @ pt_hom)[2]
        depth_b = (P_b @ pt_hom)[2]
        
        if depth_a <= 0 or depth_b <= 0:
            continue
        
        # Check reprojection
        proj_a = P_a @ pt_hom
        proj_a_2d = proj_a[:2] / proj_a[2]
        error_a = np.linalg.norm(proj_a_2d - pts_a_2d[i])
        
        proj_b = P_b @ pt_hom
        proj_b_2d = proj_b[:2] / proj_b[2]
        error_b = np.linalg.norm(proj_b_2d - pts_b_2d[i])
        
        if error_a < 5.0 and error_b < 5.0:
            valid_points.append(pt)
            valid_indices.append(i)
    
    all_triangulated[pair_name] = {
        'points_3d': np.array(valid_points),
        'valid_indices': valid_indices,
        'match_info': result['match_info'],
        'frame_a': frame_a
    }
    
    print(f"  {len(valid_points)} valid points")

total = sum(len(d['points_3d']) for d in all_triangulated.values())
print(f"\nTotal 3D points: {total} (all in Meshroom's unified coordinate system)")


Triangulating 1-5
  0 valid points

Triangulating 1-10
  49 valid points

Triangulating 5-10
  6 valid points

Triangulating 10-15
  93 valid points

Triangulating 15-20
  48 valid points

Total 3D points: 196 (all in Meshroom's unified coordinate system)


In [12]:
# Build feature database
feature_database = []

for pair_name, data in all_triangulated.items():
    frame_a = data['frame_a']
    points_3d = data['points_3d']
    valid_indices = data['valid_indices']
    match_info = data['match_info']
    
    for i, valid_idx in enumerate(valid_indices):
        # Get the match info for this valid point
        match = match_info[valid_idx]
        kp_idx_a = match['kp_idx_a']
        descriptor = features[frame_a]['descriptors'][kp_idx_a]
        
        feature_database.append({
            'descriptor': descriptor,
            'point_3d': points_3d[i]
        })

print(f"Feature database: {len(feature_database)} entries")

# Query with frame 25
query_desc = features[25]['descriptors']
query_kp = features[25]['keypoints']

db_desc = np.array([e['descriptor'] for e in feature_database])
matches = extractor.match_features(query_desc, db_desc)

print(f"Found {len(matches)} raw matches")

# Filter duplicates
best_matches = {}
for match in matches:
    query_pt = query_kp[match.queryIdx].pt
    world_pt = feature_database[match.trainIdx]['point_3d']
    key = tuple(world_pt)
    
    if key not in best_matches or match.distance < best_matches[key]['distance']:
        best_matches[key] = {'query_2d': query_pt, 'world_3d': world_pt, 'distance': match.distance}

query_2d = np.array([v['query_2d'] for v in best_matches.values()])
world_3d = np.array([v['world_3d'] for v in best_matches.values()])

print(f"Unique correspondences: {len(query_2d)}")

# Solve PnP
if len(query_2d) >= 4:
    success, rvec, tvec = cv2.solvePnP(world_3d, query_2d, K, None, flags=cv2.SOLVEPNP_ITERATIVE)
    
    if success:
        R_est, _ = cv2.Rodrigues(rvec)
        # PnP gives world-to-camera, we want camera position
        cam_center_est = -R_est.T @ tvec.flatten()
        
        meshroom_pos = frame_to_pose[25]['center']
        error = np.linalg.norm(cam_center_est - meshroom_pos)
        
        print(f"\nEstimated frame 25 position: {cam_center_est}")
        print(f"Meshroom frame 25 position:  {meshroom_pos}")
        print(f"Error: {error:.4f} meters")
    else:
        print("PnP failed")
else:
    print(f"Not enough correspondences: {len(query_2d)}")

Feature database: 196 entries
Found 295 raw matches
Unique correspondences: 19

Estimated frame 25 position: [ -4.1529214  -14.69349622  -0.09176367]
Meshroom frame 25 position:  [ -0.44401711 -14.03072321  -0.47325309]
Error: 3.7869 meters
